# Neuron reconstruction termination plot

### Objective: Convert axon termination points from swcs in a directory to volume

#### Components:

1. Read swc files and convert to graph tree object using the anytree module
2. Instantiate a blank numpy array of same size as volume (in case of full-resolution CCF volume, shape: 1320,800,1140)
3. For every point in graph tree, if that point has no children (ie. is a leaf in anytree nomenclature), plot that point in the empty array
4. Loop over all swc files in directory and iterate over same volume
5. Load in a volume mask (in this example, caudoputamen volume) and mask point cloud volume
6. Save masked point cloud volume as an *.nrrd file

##### Load required modules

In [1]:
import os
import numpy as np
from anytree import Node,RenderTree
import SimpleITK as sitk

##### Step 1: Read swc files and convert to graph tree object

Data organization here is that all swcs from a specific cortical region are in the same directory. In the end, we will have a single volume with points from all files in the directory.

Components 1-4 are done within a for loop so we'll need to get directory info and instantiate the blank numpy array before starting the loop

In [2]:
swc_path = '../data/ctx_swcs/' #swc directory path
swc_filelist = os.listdir(swc_path)

In [4]:
brain_vol = np.zeros((1320,800,1140),dtype=int) #Shape here is hard-coded but will be more flexible after refactor
resolution = 10 # microns

Loop over all files in directory to generate anytree structure.

Structure consists of Node objects that have:
1. Name
2. Parent (soma has no parent)
3. x-coordinate
4. y-coordinate
5. z-coordinate

If we want to save the anytree structure for later use, this can be done using the JSONExporter function in anytree

In [10]:
for swc_file in swc_filelist:
    
    # Read in file
    swc_db = np.genfromtxt(swc_path+swc_file)
    names=swc_db[:,0].astype(int)
    assert swc_db[0,-1]==-1 # Make sure that the first row in the last column is [-1] aka root
    
    # Start a node in the graph tree at the soma
    neuron = Node(f'p{names[0]}',
                  parent=None,
                  x=swc_db[0,2],
                  y=swc_db[0,3],
                  z=swc_db[0,4])
    
    # Iterate through all nodes to create a graph tree using dicts
    nodes={}
    nodes[neuron.name]=neuron
    for count,point in enumerate(names[1:,]):
        name = f'p{point}'
        nodes[name]=Node(
            name,
            parent=nodes[f'p{swc_db[count+1,-1].astype(int)}'],
            x=swc_db[count+1,2],
            y=swc_db[count+1,3],
            z=swc_db[count+1,4])

    # Component 3: If point has no children (ie. is a leaf in anytree nomenclature), plot that point in the empty array
    for point in nodes:
        if nodes[point].is_leaf==True:
            x = int(nodes[point].x/resolution) #Divide by resolution to fit to CCF coordinate space
            y = int(nodes[point].y/resolution)
            z = int(nodes[point].z/resolution)
            brain_vol[x,y,z]=255 

##### Step 2: Convert point cloud volume to SimpleITK image object

In [6]:
brain_vol = sitk.GetImageFromArray(brain_vol)
brain_vol = sitk.PermuteAxes(brain_vol,[2,1,0]) # This is a function of differences between how numpy orders axes [z,y,x] and how SimpleITK order axes [x,y,z]. This step flips axes to match the expected axes order in ASL orientation.

##### Step 3: Load in mask volume and use it to mask point cloud volume

In [7]:
mask_filepath = '../data/ccf_volumes/cp_mask.nrrd'

reader = sitk.ImageFileReader()
reader.SetImageIO("NrrdImageIO")
reader.SetFileName(mask_filepath)

mask = reader.Execute()
mask = sitk.Cast(mask,sitk.sitkInt32) # Convert mask file to int32 dtype for multiplication in the next step

Align point cloud volume and mask and multiply images to generate masked image

In [8]:
brain_vol.SetDirection(mask.GetDirection()) #SimpleITK is sensitive to the 'direction' of the volume even after permuting axes. This aligns point cloud and mask volumes

cp_termini = sitk.Multiply(brain_vol,mask)

##### Step 4: Save volume as an *.nrrd file

In [9]:
sitk.WriteImage(cp_termini,swc_path+'masked_point_volume.nrrd',True) # Sets compression = True